In [8]:
import os
import json
from dotenv import load_dotenv

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader
load_dotenv()
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")

# Initialize the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2,0-flash", temperature=0.2)

In [2]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

In [ ]:
pip install langchain_anthropic

In [44]:
from langchain.llms import Ollama

llm = Ollama(model="llama3")  # You can specify llama3, llama3:8b, llama2, etc.


/var/folders/y4/0lcmny8s1696ft7y4dphy7kr0000gn/T/ipykernel_20350/3301547011.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")  # You can specify llama3, llama3:8b, llama2, etc.


In [40]:
from langchain_anthropic import ChatAnthropic
api_key = os.getenv("ANTHROPIC_API_KEY")
llm = ChatAnthropic(model="claude-3-sonnet-20240229")  # Example model


In [41]:
if not api_key:
    raise ValueError("Anthropic API key not found. Check your .env file.")

# Business agent

In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",  # Full name required
    temperature=0.7
)

In [9]:

# Define the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a business consultant. Analyze the provided documents and generate 3 actionable, section-referenced suggestions to improve the business model. Focus on revenue, operational efficiency, and customer satisfaction."),
    ("user", "{context}\n\n{question}")
])

chain = LLMChain(llm=llm, prompt=prompt)

# Load and split the PDF document
def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    chunks = text_splitter.split_documents(documents)
    return chunks

# Process JSON dataset into Document chunks
def process_json_dataset(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    documents = []
    for entry in data:
        documents.append(Document(page_content=f"Brief: {entry['brief']}\nSuggestions: {entry['suggestions']}"))

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    chunks = text_splitter.split_documents(documents)
    return chunks

# Build FAISS vectorstore
def build_vectorstore(documents):
    return FAISS.from_documents(documents, embedding_model)

# Combined retrieval from both sources
def retrieve_combined_context(pdf_vectorstore, json_vectorstore, query, k=2):
    pdf_docs = pdf_vectorstore.similarity_search(query, k=k)
    json_docs = json_vectorstore.similarity_search(query, k=k)

    combined_context = "\n\n".join([doc.page_content for doc in pdf_docs + json_docs])
    return combined_context

# Business agent pipeline
def business_agent_pipeline(client_pdf_path, client_query, json_dataset_path):
    # Process client PDF
    pdf_chunks = load_and_split_pdf(client_pdf_path)
    pdf_vectorstore = build_vectorstore(pdf_chunks)

    # Process JSON dataset
    json_chunks = process_json_dataset(json_dataset_path)
    json_vectorstore = build_vectorstore(json_chunks)

    # Retrieve context
    context = retrieve_combined_context(pdf_vectorstore, json_vectorstore, client_query, k=2)

    # Run the LLM chain
    response = chain.invoke({"context": context, "question": client_query})
    return response["text"]



In [11]:

# Example usage
if __name__ == "__main__":
    client_pdf_path = "/Users/chandrimadas/Documents/agents_streamlit/sample_brief.pdf"
    json_dataset_path = "/Users/chandrimadas/Documents/agents_streamlit/business_agent_dataset_detailed.json"
    client_query = "Suggest a way to enhance patient satisfaction and retention in a healthcare mobile app, considering secure data handling and remote access features."

    suggestions = business_agent_pipeline(client_pdf_path, client_query, json_dataset_path)
    print("\n🔍 Business Agent Suggestions:\n")
    print(suggestions)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.model: unexpected model name format


# Health QA Agent

In [9]:
pip install bs4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [bs4]
Note: you may need to restart the kernel to use updated packages.


In [12]:

from langchain.document_loaders import WebBaseLoader

urls = [
    "https://www.hhs.gov/hipaa/for-professionals/privacy/index.html",
    "https://www.fda.gov/regulatory-information/search-fda-guidance-documents/mobile-medical-applications",
    "https://www.hhs.gov/ash/patient-safety/index.html"
]

# Load web pages directly
loaders = [WebBaseLoader(url) for url in urls]
docs = []
for loader in loaders:
    docs.extend(loader.load())
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = splitter.split_documents(docs)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
qa_vectordb = FAISS.from_documents(chunks, embedding=embedding_model)

# Save the vector store
qa_vectordb.save_local("./health_qa_faiss_db")
# qa_vectordb = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="./health_qa_chroma_db")

qa_retriever = qa_vectordb.as_retriever(search_kwargs={"k": 5})


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a healthcare compliance officer and technical specification preparer. Your task is to validate business suggestions for compliance with HIPAA, FDA, and HHS regulations and refine them into technically actionable specifications.\n\n"
     
     "✅ Instructions:\n"
     "- For each suggestion:\n"
     "  * If fully compliant, refine it into a detailed technical specification.\n"
     "  * If not compliant, modify it to comply.\n"
     "  * Exclude suggestions that cannot be made compliant.\n\n"
     
     "✅ Output Format:\n"
     "- Provide a numbered list of refined, technically actionable suggestions.\n"
     "- Each suggestion must:\n"
     "    • Include specific technologies, frameworks, or implementation methods.\n"
     "    • Clearly describe how to ensure regulatory compliance (e.g., encryption standard, API security protocol).\n"
    #  "    • Include regulatory tags at the end like [HIPAA], [FDA], [HHS].\n"
     "    • Be written clearly enough for a technical team to immediately start system design or development.\n"
     "- Do not include introductions or explanations. Only output the refined suggestions.\n"
    ),
    ("user", "{context}\n\nSuggestions to validate:\n{suggestions}")
])


qa_chain = LLMChain(llm=llm, prompt=qa_prompt)

def run_healthcare_qa(suggestions, retriever):
    query = "Does this comply with HIPAA, FDA, and HHS guidelines?"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    qa_response = qa_chain.invoke({"context": context, "suggestions": suggestions})
    return qa_response["text"].strip()

In [21]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
def load_healthcare_vectorstore():
    
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    # Load the FAISS vector store from the saved directory
    qa_vectordb = FAISS.load_local(
        "/Users/chandrimadas/Documents/agents_streamlit/vectorstores/healthcare_qa_faiss_db", 
        embedding_model,
        allow_dangerous_deserialization=True  # Add this if using newer versions
    )
    
    return qa_vectordb.as_retriever(search_kwargs={"k": 5})

In [22]:
qa_retriever = load_healthcare_vectorstore()

In [40]:

qa_result = run_healthcare_qa(suggestions, qa_retriever)

print("\n✅ QA Agent Feedback:\n")
print(qa_result)



✅ QA Agent Feedback:

1. **Enhanced Revenue Streams through Value-Added Services:**

* **Technical Specification:** Develop a tiered subscription model for the mobile application.  The premium tier will include:
    * **Personalized Health Coaching:** Integrate with a third-party HIPAA-compliant health coaching platform via a secure API (using OAuth 2.0 for authorization).  Implement role-based access control (RBAC) to restrict access to patient data based on clinician and coach roles.  All communications will be encrypted using TLS 1.3 or higher.
    * **Advanced Analytics and Reporting:** Utilize a HIPAA-compliant data analytics platform to generate personalized reports for patients and clinicians.  Data aggregation and analysis will be performed on de-identified data whenever possible.  Reports will be accessible via secure, password-protected user accounts with multi-factor authentication (MFA).  Data encryption at rest and in transit will be implemented using AES-256.
    * **Wea

# Edu QA Agent

In [ ]:

from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

# URLs for Education Compliance
edu_urls = [
    "https://studentprivacy.ed.gov/",
    "https://www.ftc.gov/business-guidance/privacy-security/childrens-privacy",
    "https://www.ada.gov/resources/overview/"
]

# Load web pages
edu_loaders = [WebBaseLoader(url) for url in edu_urls]
edu_docs = []
for loader in edu_loaders:
    edu_docs.extend(loader.load())

# Split documents
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
edu_chunks = splitter.split_documents(edu_docs)

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build Education Vector DB
edu_qa_vectordb = Chroma.from_documents(edu_chunks, embedding=embedding_model, persist_directory="./edu_qa_chroma_db")

# Build Retriever
edu_qa_retriever = edu_qa_vectordb.as_retriever(search_kwargs={"k": 5})

# Education QA Prompt
edu_qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an education compliance officer and technical specification preparer. Validate business suggestions for FERPA, COPPA, and ADA compliance, and refine them into technically actionable specifications.\n\n"
     "✅ Instructions:\n"
     "- For each suggestion:\n"
     "  * If fully compliant, refine it into a detailed technical specification.\n"
     "  * If not compliant, modify it to comply.\n"
     "  * Exclude suggestions that cannot be made compliant.\n\n"
     "✅ Output Format:\n"
     "- Provide a numbered list of refined, technically actionable suggestions.\n"
     "- Each must:\n"
     "    • Include technologies, frameworks, and implementation methods.\n"
     "    • Clearly describe how to ensure FERPA, COPPA, and ADA compliance.\n"
     "- Do not include introductions or explanations. Only output the refined suggestions.\n"
    ),
    ("user", "{context}\n\nSuggestions to validate:\n{suggestions}")
])

edu_qa_chain = LLMChain(llm=llm, prompt=edu_qa_prompt)

def run_education_qa(suggestions, retriever):
    query = "Does this comply with FERPA, COPPA, and ADA regulations?"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    edu_response = edu_qa_chain.invoke({"context": context, "suggestions": suggestions})
    return edu_response["text"].strip()


# General qa agent

In [ ]:

# URLs for General Compliance
general_urls = [
    "https://gdpr-info.eu/",
    "https://oag.ca.gov/privacy/ccpa",
    "https://www.ftc.gov/business-guidance/privacy-security"
]

# Load web pages
general_loaders = [WebBaseLoader(url) for url in general_urls]
general_docs = []
for loader in general_loaders:
    general_docs.extend(loader.load())

# Split documents
general_chunks = splitter.split_documents(general_docs)

# Build General Vector DB
general_qa_vectordb = Chroma.from_documents(general_chunks, embedding=embedding_model, persist_directory="./general_qa_chroma_db")

# Build Retriever
general_qa_retriever = general_qa_vectordb.as_retriever(search_kwargs={"k": 5})

# General QA Prompt
general_qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a compliance officer for general business regulations. Validate business suggestions for GDPR, CCPA, and FTC compliance, and refine them into technically actionable specifications.\n\n"
     "✅ Instructions:\n"
     "- For each suggestion:\n"
     "  * If fully compliant, refine it into a detailed technical specification.\n"
     "  * If not compliant, modify it to comply.\n"
     "  * Exclude suggestions that cannot be made compliant.\n\n"
     "✅ Output Format:\n"
     "- Provide a numbered list of refined, technically actionable suggestions.\n"
     "- Each must:\n"
     "    • Include technologies, frameworks, and implementation methods.\n"
     "    • Clearly describe how to ensure GDPR, CCPA, and FTC compliance.\n"
     "- Do not include introductions or explanations. Only output the refined suggestions.\n"
    ),
    ("user", "{context}\n\nSuggestions to validate:\n{suggestions}")
])

general_qa_chain = LLMChain(llm=llm, prompt=general_qa_prompt)

def run_general_qa(suggestions, retriever):
    query = "Does this comply with GDPR, CCPA, and FTC regulations?"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    general_response = general_qa_chain.invoke({"context": context, "suggestions": suggestions})
    return general_response["text"].strip()


In [24]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from pathlib import Path
import os

# === STEP 1: All documentation URLs by category ===
documentation_sources = {
    "firebase": [
        "https://firebase.google.com/docs/auth",
        "https://firebase.google.com/docs/firestore/security/get-started",
        "https://firebase.google.com/docs/functions",
        "https://firebase.google.com/docs/cloud-messaging",
        "https://firebase.google.com/docs/emulator-suite"
    ],
    "gcp": [
        "https://cloud.google.com/iam/docs/understanding-roles",
        "https://cloud.google.com/iam/docs/service-accounts",
        "https://cloud.google.com/resource-manager/docs/access-control-org"
    ],
    "mongodb": [
        "https://www.mongodb.com/docs/manual/core/data-model-design/",
        "https://www.mongodb.com/docs/manual/core/index-types/",
        "https://www.mongodb.com/docs/manual/core/security-users/",
        "https://www.mongodb.com/docs/manual/core/aggregation-pipeline/",
        "https://www.mongodb.com/docs/manual/tutorial/manage-users/"
    ],
    "react_native": [
        "https://docs.expo.dev/versions/latest/sdk/ble/",
        "https://docs.expo.dev/versions/latest/sdk/camera/",
        "https://docs.expo.dev/push-notifications/overview/"
    ],
    "ble_beacons": [
        "https://developer.estimote.com/proximity/",
        "https://developer.estimote.com/indoor/",
        "https://developer.kontakt.io/"
    ],
    "pos_integration": [
        "https://developer.shopify.com/docs/api",
        "https://developer.squareup.com/reference/square",
        "https://developer.vendhq.com/documentation/introduction"
    ]
}

# === STEP 2: Config ===
PERSIST_DIR = "./full_tech_vectorstore"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

# === STEP 3: Load, Chunk, Embed, and Store ===
def build_vectorstore(doc_sources):
    all_chunks = []

    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    for category, urls in doc_sources.items():
        print(f"\n📥 Loading category: {category}")
        try:
            loader = WebBaseLoader(urls)
            docs = loader.load()
            for doc in docs:
                doc.metadata["category"] = category
            chunks = splitter.split_documents(docs)
            for chunk in chunks:
                chunk.metadata["category"] = category
            all_chunks.extend(chunks)
        except Exception as e:
            print(f"⚠️ Error loading {category}: {e}")

    print("\n🧠 Embedding and saving to vectorstore...")
    tech_vectordb = FAISS.from_documents(all_chunks, embedding=embedding_model)
    tech_vectordb.save_local("/Users/chandrimadas/Documents/agents_streamlit/vectorstores/tech_faiss_db")

    # vectordb = Chroma.from_documents(all_chunks, embedding=embedding_model, persist_directory=persist_dir)
    # vectordb.persist()
    # print(f"\n✅ Vectorstore saved to {persist_dir}")

# === Run the builder ===
if __name__ == "__main__":
    # Path(PERSIST_DIR).mkdir(exist_ok=True)
    build_vectorstore(documentation_sources)



📥 Loading category: firebase

📥 Loading category: gcp

📥 Loading category: mongodb

📥 Loading category: react_native

📥 Loading category: ble_beacons

📥 Loading category: pos_integration

🧠 Embedding and saving to vectorstore...


In [47]:

from langchain.vectorstores import Chroma
# from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Load your tech vector store
tech_vectordb = Chroma(persist_directory="/Users/chandrimadas/Documents/dual_agents/full_tech_vectorstore", embedding_function=embedding_model)

# Initialize Gemini model
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Define technical agent prompt
tech_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a technical solutions architect. Based on the provided business suggestions and technical documentation, provide detailed, step-by-step implementation strategies, tools, and best practices. Be specific and include architecture, services, and data considerations."),
    ("user", "{tech_context}\n\nBusiness Suggestions: {business_suggestions}")
])

# Set up the LLM chain for the technical agent
tech_llm_chain = LLMChain(llm=chat_model, prompt=tech_prompt)


# Function to retrieve technical context
def retrieve_tech_context(vectorstore, business_suggestions, k=3):
    docs = vectorstore.similarity_search(business_suggestions, k=k)
    return "\n\n".join([doc.page_content for doc in docs])


# Complete technical agent pipeline
def technical_agent_pipeline(business_suggestions):
    # Retrieve the most relevant technical docs
    tech_context = retrieve_tech_context(tech_vectordb, business_suggestions, k=3)
    
    # Get implementation strategies from Gemini
    response = tech_llm_chain.run(tech_context=tech_context, business_suggestions=business_suggestions)
    
    return response





In [49]:
# Example usage
if __name__ == "__main__":
    # Example validated suggestions from QA agent
    validated_suggestions = """
    
    1. Implement an intuitive, voice-assisted mobile app to support elderly patients.
    2. Integrate real-time appointment reminders via SMS and automated calls.
    # 3. Ensure HIPAA-compliant cloud storage for patient data and records.
    """

    technical_solution = technical_agent_pipeline(qa_result)
    print("Technical Implementation Plan:\n")
    print(technical_solution)

Technical Implementation Plan:

## Implementation Strategy: Proactive Customer Support for Square

This document outlines a step-by-step implementation strategy for proactive customer support, incorporating in-app help, HIPAA-compliant live chat, and secure logging.

**I. Architecture:**

The solution will utilize a microservices architecture to ensure scalability, maintainability, and flexibility. Key services include:

* **In-App Help Center Service:**  This service will handle the delivery of FAQs, tutorials, and contextual help within the Square application.  It will leverage a content management system (CMS) for easy updates and version control.
* **Live Chat Service:** This service will integrate with a HIPAA-compliant live chat platform (e.g.,  Talkdesk, Zendesk, or a custom solution built on a compliant platform like AWS HIPAA eligible services).  It will manage real-time chat sessions, agent routing, and chat history.
* **Logging and Analytics Service:** This service will secu

In [75]:

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Categorization prompt for the entire suggestion block
categorization_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a classification engine. Categorize the entire set of business suggestions into one of the following domains:\n\n"
     "- Healthcare\n- Education\n- General\n\n"
     "✅ Instructions:\n"
     "Analyze the suggestions and return only one category name based on the primary focus.\n\n"
     "✅ Output Format:\n"
     "Healthcare\nOR\nEducation\nOR\nGeneral"),
    ("user", "Business Suggestions:\n{suggestions}")
])

categorization_chain = LLMChain(llm=llm, prompt=categorization_prompt)

def categorize_suggestions(suggestions):
    result = categorization_chain.invoke({"suggestions": suggestions})
    return result['text'].strip()


In [76]:
def route_suggestions(suggestions, retrievers):
    category = categorize_suggestions(suggestions)

    if category == "Healthcare":
        result = run_healthcare_qa(suggestions, retrievers['healthcare'])
    elif category == "Education":
        result = run_education_qa(suggestions, retrievers['education'])
    elif category == "General":
        result = run_general_qa(suggestions, retrievers['general'])
    else:
        result = "❌ Could not determine category. Please review suggestions."

    return result


In [77]:
if __name__ == "__main__":
    retrievers = {
        "healthcare": qa_retriever,
        "education": edu_qa_retriever,  # Pre-built similar to qa_retriever but trained on FERPA, COPPA, ADA
        "general": general_qa_retriever  # Pre-built similar to qa_retriever but trained on GDPR, CCPA, FTC
    }

    validated_suggestions = """
    1. Implement an intuitive, voice-assisted mobile app to support elderly patients.
    2. Integrate student behavior tracking dashboards in K-12 educational apps.
    3. Set up a global user tracking system for personalized e-commerce recommendations.
    """

    qa_result = route_suggestions(suggestions, retrievers)

    print("\n✅ Categorized & QA Agent Feedback:\n")
    print(qa_result)

    technical_solution = technical_agent_pipeline(qa_result)
    print("\n🛠️ Technical Implementation Plan:\n")
    print(technical_solution)



✅ Categorized & QA Agent Feedback:

1. **Tiered Subscription Model with Value-Added Services:**

* **Technology:**  Develop a microservices architecture using Node.js/Express.js for backend APIs and React Native for cross-platform mobile app development.  Implement a robust payment gateway integration (e.g., Stripe, Braintree) adhering to PCI DSS standards.  Utilize a cloud-based database (e.g., AWS RDS, Google Cloud SQL) with encryption at rest and in transit (AES-256).
* **Compliance:**  All data transmission will be encrypted using TLS 1.3 or higher.  HIPAA compliance will be achieved through adherence to the Security Rule, including access controls, audit trails, and data integrity mechanisms.  PHI will be stored and processed according to HIPAA regulations.  Telehealth consultations will comply with relevant state and federal regulations.
* **Implementation:**  Develop three subscription tiers: Basic (free, core features), Premium (paid, telehealth consultations, personalized hea

In [58]:
print(qa_result)

In [ ]:

# Define the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a business consultant. Analyze the provided documents and generate 3 actionable, section-referenced suggestions to improve the business model. Focus on revenue, operational efficiency, and customer satisfaction."),
    ("user", "{context}\n\n{question}")
])

chain = LLMChain(llm=llm, prompt=prompt)

# Load and split the PDF document
def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    chunks = text_splitter.split_documents(documents)
    return chunks

# Process JSON dataset into Document chunks
def process_json_dataset(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    documents = []
    for entry in data:
        documents.append(Document(page_content=f"Brief: {entry['brief']}\nSuggestions: {entry['suggestions']}"))

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    chunks = text_splitter.split_documents(documents)
    return chunks

# Build FAISS vectorstore
def build_vectorstore(documents):
    return FAISS.from_documents(documents, embedding_model)

# Combined retrieval from both sources
def retrieve_combined_context(pdf_vectorstore, json_vectorstore, query, k=2):
    pdf_docs = pdf_vectorstore.similarity_search(query, k=k)
    json_docs = json_vectorstore.similarity_search(query, k=k)

    combined_context = "\n\n".join([doc.page_content for doc in pdf_docs + json_docs])
    return combined_context

# Business agent pipeline
def business_agent_pipeline(client_pdf_path, client_query, json_dataset_path):
    # Process client PDF
    pdf_chunks = load_and_split_pdf(client_pdf_path)
    pdf_vectorstore = build_vectorstore(pdf_chunks)

    # Process JSON dataset
    json_chunks = process_json_dataset(json_dataset_path)
    json_vectorstore = build_vectorstore(json_chunks)

    # Retrieve context
    context = retrieve_combined_context(pdf_vectorstore, json_vectorstore, client_query, k=2)

    # Run the LLM chain
    response = chain.invoke({"context": context, "question": client_query})
    return response["text"]


# Example usage
if __name__ == "__main__":
    client_pdf_path = "/Users/chandrimadas/Documents/dual_agents/sample_brief.pdf"
    json_dataset_path = "/Users/chandrimadas/Documents/dual_agents/business_agent_dataset_detailed.json"
    client_query = "Suggest a way to enhance patient satisfaction and retention in a healthcare mobile app, considering secure data handling and remote access features."

    suggestions = business_agent_pipeline(client_pdf_path, client_query, json_dataset_path)
    print("\n🔍 Business Agent Suggestions:\n")
    print(suggestions)
    
#HEALTH QA
from langchain.document_loaders import WebBaseLoader

urls = [
    "https://www.hhs.gov/hipaa/for-professionals/privacy/index.html",
    "https://www.fda.gov/regulatory-information/search-fda-guidance-documents/mobile-medical-applications",
    "https://www.hhs.gov/ash/patient-safety/index.html"
]

# Load web pages directly
loaders = [WebBaseLoader(url) for url in urls]
docs = []
for loader in loaders:
    docs.extend(loader.load())
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = splitter.split_documents(docs)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
qa_vectordb = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="./qa_chroma_db")

qa_retriever = qa_vectordb.as_retriever(search_kwargs={"k": 5})
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a healthcare compliance officer and technical specification preparer. Your task is to validate business suggestions for compliance with HIPAA, FDA, and HHS regulations and refine them into technically actionable specifications.\n\n"
     
     "✅ Instructions:\n"
     "- For each suggestion:\n"
     "  * If fully compliant, refine it into a detailed technical specification.\n"
     "  * If not compliant, modify it to comply.\n"
     "  * Exclude suggestions that cannot be made compliant.\n\n"
     
     "✅ Output Format:\n"
     "- Provide a numbered list of refined, technically actionable suggestions.\n"
     "- Each suggestion must:\n"
     "    • Include specific technologies, frameworks, or implementation methods.\n"
     "    • Clearly describe how to ensure regulatory compliance (e.g., encryption standard, API security protocol).\n"
    #  "    • Include regulatory tags at the end like [HIPAA], [FDA], [HHS].\n"
     "    • Be written clearly enough for a technical team to immediately start system design or development.\n"
     "- Do not include introductions or explanations. Only output the refined suggestions.\n"
    ),
    ("user", "{context}\n\nSuggestions to validate:\n{suggestions}")
])

qa_chain = LLMChain(llm=llm, prompt=qa_prompt)

def run_healthcare_qa(suggestions, retriever):
    query = "Does this comply with HIPAA, FDA, and HHS guidelines?"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    qa_response = qa_chain.invoke({"context": context, "suggestions": suggestions})
    return qa_response["text"].strip()
qa_result = run_healthcare_qa(suggestions, qa_retriever)

print("\n✅ QA Agent Feedback:\n")
print(qa_result)

from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

# URLs for Education Compliance
edu_urls = [
    "https://studentprivacy.ed.gov/",
    "https://www.ftc.gov/business-guidance/privacy-security/childrens-privacy",
    "https://www.ada.gov/resources/overview/"
]

# Load web pages
edu_loaders = [WebBaseLoader(url) for url in edu_urls]
edu_docs = []
for loader in edu_loaders:
    edu_docs.extend(loader.load())

# Split documents
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
edu_chunks = splitter.split_documents(edu_docs)

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build Education Vector DB
edu_qa_vectordb = Chroma.from_documents(edu_chunks, embedding=embedding_model, persist_directory="./edu_qa_chroma_db")

# Build Retriever
edu_qa_retriever = edu_qa_vectordb.as_retriever(search_kwargs={"k": 5})

# Education QA Prompt
edu_qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an education compliance officer and technical specification preparer. Validate business suggestions for FERPA, COPPA, and ADA compliance, and refine them into technically actionable specifications.\n\n"
     "✅ Instructions:\n"
     "- For each suggestion:\n"
     "  * If fully compliant, refine it into a detailed technical specification.\n"
     "  * If not compliant, modify it to comply.\n"
     "  * Exclude suggestions that cannot be made compliant.\n\n"
     "✅ Output Format:\n"
     "- Provide a numbered list of refined, technically actionable suggestions.\n"
     "- Each must:\n"
     "    • Include technologies, frameworks, and implementation methods.\n"
     "    • Clearly describe how to ensure FERPA, COPPA, and ADA compliance.\n"
     "- Do not include introductions or explanations. Only output the refined suggestions.\n"
    ),
    ("user", "{context}\n\nSuggestions to validate:\n{suggestions}")
])

edu_qa_chain = LLMChain(llm=llm, prompt=edu_qa_prompt)

def run_education_qa(suggestions, retriever):
    query = "Does this comply with FERPA, COPPA, and ADA regulations?"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    edu_response = edu_qa_chain.invoke({"context": context, "suggestions": suggestions})
    return edu_response["text"].strip()

# URLs for General Compliance
general_urls = [
    "https://gdpr-info.eu/",
    "https://oag.ca.gov/privacy/ccpa",
    "https://www.ftc.gov/business-guidance/privacy-security"
]

# Load web pages
general_loaders = [WebBaseLoader(url) for url in general_urls]
general_docs = []
for loader in general_loaders:
    general_docs.extend(loader.load())

# Split documents
general_chunks = splitter.split_documents(general_docs)

# Build General Vector DB
general_qa_vectordb = Chroma.from_documents(general_chunks, embedding=embedding_model, persist_directory="./general_qa_chroma_db")

# Build Retriever
general_qa_retriever = general_qa_vectordb.as_retriever(search_kwargs={"k": 5})

# General QA Prompt
general_qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a compliance officer for general business regulations. Validate business suggestions for GDPR, CCPA, and FTC compliance, and refine them into technically actionable specifications.\n\n"
     "✅ Instructions:\n"
     "- For each suggestion:\n"
     "  * If fully compliant, refine it into a detailed technical specification.\n"
     "  * If not compliant, modify it to comply.\n"
     "  * Exclude suggestions that cannot be made compliant.\n\n"
     "✅ Output Format:\n"
     "- Provide a numbered list of refined, technically actionable suggestions.\n"
     "- Each must:\n"
     "    • Include technologies, frameworks, and implementation methods.\n"
     "    • Clearly describe how to ensure GDPR, CCPA, and FTC compliance.\n"
     "- Do not include introductions or explanations. Only output the refined suggestions.\n"
    ),
    ("user", "{context}\n\nSuggestions to validate:\n{suggestions}")
])

general_qa_chain = LLMChain(llm=llm, prompt=general_qa_prompt)

def run_general_qa(suggestions, retriever):
    query = "Does this comply with GDPR, CCPA, and FTC regulations?"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    general_response = general_qa_chain.invoke({"context": context, "suggestions": suggestions})
    return general_response["text"].strip()

from langchain.vectorstores import Chroma
# from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Load your tech vector store
tech_vectordb = Chroma(persist_directory="/Users/chandrimadas/Documents/dual_agents/full_tech_vectorstore", embedding_function=embedding_model)

# Initialize Gemini model
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Define technical agent prompt
tech_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a technical solutions architect. Based on the provided business suggestions and technical documentation, provide detailed, step-by-step implementation strategies, tools, and best practices. Be specific and include architecture, services, and data considerations."),
    ("user", "{tech_context}\n\nBusiness Suggestions: {business_suggestions}")
])

# Set up the LLM chain for the technical agent
tech_llm_chain = LLMChain(llm=chat_model, prompt=tech_prompt)


# Function to retrieve technical context
def retrieve_tech_context(vectorstore, business_suggestions, k=3):
    docs = vectorstore.similarity_search(business_suggestions, k=k)
    return "\n\n".join([doc.page_content for doc in docs])


# Complete technical agent pipeline
def technical_agent_pipeline(business_suggestions):
    # Retrieve the most relevant technical docs
    tech_context = retrieve_tech_context(tech_vectordb, business_suggestions, k=3)
    
    # Get implementation strategies from Gemini
    response = tech_llm_chain.run(tech_context=tech_context, business_suggestions=business_suggestions)
    
    return response



from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Categorization prompt for the entire suggestion block
categorization_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a classification engine. Categorize the entire set of business suggestions into one of the following domains:\n\n"
     "- Healthcare\n- Education\n- General\n\n"
     "✅ Instructions:\n"
     "Analyze the suggestions and return only one category name based on the primary focus.\n\n"
     "✅ Output Format:\n"
     "Healthcare\nOR\nEducation\nOR\nGeneral"),
    ("user", "Business Suggestions:\n{suggestions}")
])

categorization_chain = LLMChain(llm=llm, prompt=categorization_prompt)

def categorize_suggestions(suggestions):
    result = categorization_chain.invoke({"suggestions": suggestions})
    return result['text'].strip()

def route_suggestions(suggestions, retrievers):
    category = categorize_suggestions(suggestions)

    if category == "Healthcare":
        result = run_healthcare_qa(suggestions, retrievers['healthcare'])
    elif category == "Education":
        result = run_education_qa(suggestions, retrievers['education'])
    elif category == "General":
        result = run_general_qa(suggestions, retrievers['general'])
    else:
        result = "❌ Could not determine category. Please review suggestions."

    return result
if __name__ == "__main__":
    retrievers = {
        "healthcare": qa_retriever,
        "education": edu_qa_retriever,  # Pre-built similar to qa_retriever but trained on FERPA, COPPA, ADA
        "general": general_qa_retriever  # Pre-built similar to qa_retriever but trained on GDPR, CCPA, FTC
    }

    validated_suggestions = """
    1. Implement an intuitive, voice-assisted mobile app to support elderly patients.
    2. Integrate student behavior tracking dashboards in K-12 educational apps.
    3. Set up a global user tracking system for personalized e-commerce recommendations.
    """

    qa_result = route_suggestions(suggestions, retrievers)

    print("\n✅ Categorized & QA Agent Feedback:\n")
    print(qa_result)

    technical_solution = technical_agent_pipeline(qa_result)
    print("\n🛠️ Technical Implementation Plan:\n")
    print(technical_solution)
